# imports

https://www.wrl.unsw.edu.au/news/coastsat-how-to-estimate-beach-slopes-using-satellite-imagery
https://www.wrl.unsw.edu.au/research/coastsat


In [17]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pickle
import warnings
import time
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.ion()
import pandas as pd
from datetime import datetime
from coastsat import SDS_earthcache_api, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects, SDS_earthcache_client

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## setup timeline and AOL

In [9]:
#setup monthly pipelines
def fmt(x):
    if x < 10:
        return '0' + str(x)
    else:
        return str(x)
def monthly_pipes(startyear,endyear=2024,day=[0,0]):  
    starts = [] ; ends = []
    dates = range(startyear, endyear)  
    for y in dates:
        for m in range(1, 13):
            if day[0] == 0:
                starts.append(str(y) + '-' + fmt(m) + '-01')
            else:
                starts.append(str(y) + '-' + fmt(m) + '-' + fmt(day[0]))

            if day[1] == 0:
                if m == 2:
                    ends.append(str(y) + '-' + fmt(m) + '-28')
                elif m in [4, 6, 9, 11]:
                    ends.append(str(y) + '-' + fmt(m) + '-30')
                else:
                    ends.append(str(y) + '-' + fmt(m) + '-31')
            else:
                ends.append(str(y) + '-' + fmt(m) + '-' + fmt(day[1]))
    return starts, ends

In [10]:
# these coordinates are a rectangle around the island of Madhirivaadhoo
aoi = { "type": "Polygon", 
        "coordinates": [
      [
          [
            72.85318351534859,
            4.227805672204206,
            0
          ],
          [
            72.85339136125246,
            4.207331395520955,
            0
          ],
          [
            72.87337028624503,
            4.207517991124759,
            0
          ],
          [
            72.8731121458676,
            4.227805397834814,
            0
          ],
          [
            72.85318351534859,
            4.227805672204206,
            0
          ]
      ]
    ]
    }

# setup pipeline

In [11]:
key = input("please enter your API key for Earthcache:")

In [19]:
nb_client =SDS_earthcache_api.initialize_client(key,max_cost=100)
location_name = 'Ukulhas_'
resolution = 'medium'
interval = '5d'

d5431cc4-b43b-4f31-8db4-d1efe9e589c9


In [ ]:
#nb_client.getOutputs()

(200,
 {'data': [{'id': '154311a8-582a-11e7-b30d-7291b81e23e1',
    'name': 'NDVI',
    'description': 'Normalized Difference Vegetation Index (NDVI) images is a ratio image of the near-infrared, and red bands. NDVI images can be used to measure biomass, assess plant health and stress levels, quantify forest supply, and monitor plant growth. By combining NDVI data with precipitation data, it is also possible to identify at-risk areas for drought.',
    'cover_image': {'url': 'https://s3-us-west-2.amazonaws.com/skywatch-public-imagery/outputs/NDVI.jpg',
     'alt_text': 'SkyWatch Space Applications'},
    'provider': {'name': 'SkyWatch', 'logo_url': None},
    'private': False},
   {'id': '154311a8-582a-11e7-b30d-7291b81e23e3',
    'name': 'False Colour Infrared',
    'description': 'A false colour infrared composite image is a combination of near-infrared, red, and green  bands. It is often used to track vegetation health or help identify different types of soil and minerals.',
    'co

'id': 'c31c1bea-a4be-11e7-8650-3ae5c7149ea7'

'name': 'All Optical Bands'

'description': 'All optical spectral bands in a product',

In [20]:
image_type = 'c31c1bea-a4be-11e7-8650-3ae5c7149ea7'#"a8fc3dde-a3e8-11e7-9793-ae4260ee3b4b" 

# test pipeline

In [35]:
# need to initialize the api first
#SDS_earthcache_api.initialize(key)
#create a set of dates so each pipeline is one month
starts,ends = monthly_pipes(2022)#,day=[2,0])

month_range = range(1,12) # testing with one month at a time -- change to len(starts) to run all months

In [36]:

for i in month_range:
    start_date = starts[i] 
    end_date = ends[i]
    pipeline_name = location_name + start_date + '_' + resolution
    print(pipeline_name + " ending: " + end_date)
    status, response = nb_client.calculatePrice(resolution, aoi, start_date, end_date,interval)
    if(status != 200):
        print(response['errors'])
    else:
        print(response['data']['archive'])
        print("balance: "+str(response['data']['prepurchase_balance']))
    #add 3 seconds of delay between each request
    time.sleep(3)

Ukulhas_2022-02-01_medium ending: 2022-02-28
{'interval_quantity': 6, 'price_km2': 2.5, 'minimum_area_km2': 0.1, 'interval_cost': 12.57, 'max_cost': 75.39}
balance: 3000.0
Ukulhas_2022-03-01_medium ending: 2022-03-31
{'interval_quantity': 7, 'price_km2': 2.5, 'minimum_area_km2': 0.1, 'interval_cost': 12.57, 'max_cost': 87.96}
balance: 3000.0
Ukulhas_2022-04-01_medium ending: 2022-04-30
{'interval_quantity': 6, 'price_km2': 2.5, 'minimum_area_km2': 0.1, 'interval_cost': 12.57, 'max_cost': 75.39}
balance: 3000.0
Ukulhas_2022-05-01_medium ending: 2022-05-31
{'interval_quantity': 7, 'price_km2': 2.5, 'minimum_area_km2': 0.1, 'interval_cost': 12.57, 'max_cost': 87.96}
balance: 3000.0
Ukulhas_2022-06-01_medium ending: 2022-06-30
{'interval_quantity': 6, 'price_km2': 2.5, 'minimum_area_km2': 0.1, 'interval_cost': 12.57, 'max_cost': 75.39}
balance: 3000.0
Ukulhas_2022-07-01_medium ending: 2022-07-31
{'interval_quantity': 7, 'price_km2': 2.5, 'minimum_area_km2': 0.1, 'interval_cost': 12.57, 'ma

# run pipeline

In [37]:
for i in month_range:
    start_date = starts[i]
    end_date = ends[i]
    pipeline_name = location_name + start_date + '_' + resolution
    #SDS_earthcache_api.retrieve_images_earthcache(nb_client, pipeline_name, aoi, start_date, end_date, image_type,interval,resolution)
    time.sleep(3)

201 {'data': {'id': 'b9a55c6a-071b-11ef-91f3-093f025a67e0', 'name': 'Ukulhas_2022-02-01_medium', 'start_date': '2022-02-01', 'end_date': '2022-02-28', 'aoi': {'type': 'Polygon', 'coordinates': [[[72.85318351534859, 4.227805672204206, 0], [72.85339136125246, 4.207331395520955, 0], [72.87337028624503, 4.207517991124759, 0], [72.8731121458676, 4.227805397834814, 0], [72.85318351534859, 4.227805672204206, 0]]]}, 'area_km2': 5.026146507863212, 'cloud_cover_percentage': 25, 'min_aoi_coverage_percentage': 75, 'interval': '5d', 'resolution_low': 30, 'resolution_high': 1.5, 'status': 'pending', 'tags': None, 'created_at': '2024-04-30T18:01:53.173579Z', 'updated_at': '2024-04-30T18:01:53.173579Z', 'max_cost': 100, 'result_delivery': {'max_latency': '0d', 'priorities': ['lowest_cost']}, 'output': {'id': 'c31c1bea-a4be-11e7-8650-3ae5c7149ea7', 'format': 'geotiff', 'mosaic': 'stitched'}}}
200 {'data': {'id': 'b9a55c6a-071b-11ef-91f3-093f025a67e0', 'org_id': '001JR00000AfBY1YAN', 'name': 'Ukulhas_20

# check results and download

In [42]:
# after about 20 minutes, make sure the status is ready
for i in month_range:
    start_date = starts[i]
    end_date = ends[i]
    pipeline_name = location_name + start_date + '_' + resolution
    status,res = SDS_earthcache_api.checkStatus(nb_client, pipeline_name)
    print(status,res['data'][0]['status'])

200 complete
200 complete
200 complete
200 complete
200 complete
200 complete
200 complete
200 complete
200 processing
200 complete
200 complete


In [34]:
# once the status is good, run this cell to download the images
images = []
for i in month_range:
    start_date = starts[i]
    pipeline_name = location_name + start_date + '_' + resolution
    status,res = SDS_earthcache_api.checkStatus(nb_client, pipeline_name)
    if(status == 200):
        images.append(SDS_earthcache_api.download_images(nb_client, pipeline_name))

data/Ukulhas_2022-01-01_medium\e53396ea-0889-4f17-874c-7008d21a1813
... downloading https://s3-us-west-2.amazonaws.com/aoi-processed-images-prod/0dc2b6e7-0719-11ef-92d6-093f025a67e0/f0910b42-ba53-4d92-90a3-9bc7fa7b60c9/metadata/SKYWATCH_S2_MS_20220101T0522_ALL_Tile_0_0_14ea_metadata.json
... downloading https://s3-us-west-2.amazonaws.com/aoi-processed-images-prod/0dc2b6e7-0719-11ef-92d6-093f025a67e0/f0910b42-ba53-4d92-90a3-9bc7fa7b60c9/analytic/SKYWATCH_S2_MS_20220101T0522_ALL_Tile_0_0_14ea.tif
data/Ukulhas_2022-01-01_medium\a8cd4533-9d7e-4454-a304-cff91cd97f2b
data/Ukulhas_2022-01-01_medium\b4e6f2eb-4d48-44f5-8462-2140ea594000
data/Ukulhas_2022-01-01_medium\664564cf-d02f-4b21-a45f-cfe95059cd45
data/Ukulhas_2022-01-01_medium\2f6b79c3-94dd-4b9d-b729-0151e5592d25
data/Ukulhas_2022-01-01_medium\9091064b-71ee-4b1f-9ba8-e52e9151a691
... downloading https://s3-us-west-2.amazonaws.com/aoi-processed-images-prod/0dc2b6e7-0719-11ef-92d6-093f025a67e0/adb2418e-41c5-4e58-951d-1e4ac9b0c2d0/metadata/

In [43]:
images[0][0][0]

'data/Ukulhas_2023-01-02_medium\\45bba2a6-0e36-47d8-b053-c5683e5ac7c6\\20230103_043520/SKYWATCH_PLSD_PS_20230103T0435_TC_Tile_0_0_ec9f.tif'

# visualize sample

In [ ]:
# this function visualizes the first image
SDS_earthcache_api.view_first_image(images[0])